In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [ ]:
train = pd.read_csv('train.csv')

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
train.corr()

* Seperating Categorical and Numerical varabels

In [ ]:
train.PassengerId.isnull().sum()

* Removing this column

In [ ]:
#train.Name.unique()

In [ ]:
train.Pclass.value_counts()

* Its a categorical data we need to change

In [ ]:
train.SibSp.value_counts()

In [ ]:
train.SibSp.unique()

In [ ]:
train.SibSp.value_counts()

In [ ]:
sns.histplot(train.SibSp)

In [ ]:
train.Parch.value_counts()

In [ ]:
train.Parch.unique()

In [ ]:
train.Fare.value_counts()

In [ ]:
train.Fare.unique()

In [ ]:
sns.histplot(train.Fare)

# Seperating Numerical and Categorical data

In [ ]:
# Nuumeric values
def_num = train[['Age','SibSp','Parch','Fare']]

In [ ]:
# Categorical values
def_cat = train[['Survived','Pclass','Sex','Ticket','Cabin','Embarked']]

In [ ]:
def_num.corr()

In [ ]:
for i in def_num.columns:
    plt.hist(def_num[i])
    plt.title(i)
    plt.show()

In [ ]:
def_num.isnull().sum()

In [ ]:
def_num.info()

## First checking on Continues data
* Age and Fare

In [ ]:
train.Age.isnull().sum()

In [ ]:
sns.distplot(train.Age)

In [ ]:
#pd.cut(train['Age'],10)

In [ ]:
sns.distplot(train.Fare.fillna(train.Fare.mean()))

* Right Skew data

In [ ]:
def_num['Age'] = def_num['Age'].fillna(def_num.Age.mean())

In [ ]:
def_num.isnull().sum()

In [ ]:
X = def_num[['Age','Fare']]
y = train['Survived']
X.isnull().sum()

In [ ]:
y.isnull().sum()

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer

In [ ]:
# appling on 'age' column
kbin_age = KBinsDiscretizer(n_bins=15,encode='ordinal',strategy='quantile')
# appling on 'fare' Column
kbin_fare = KBinsDiscretizer(n_bins=15,encode='ordinal',strategy='quantile')


trf = ColumnTransformer([
    ('first',kbin_age,[0]),
    ('second',kbin_fare,[1])
])

In [ ]:
# Another method to apply bins
# Add Fare to sub dataframes
df_bin  =  X['Fare'] 
df_bin = pd.cut(train['Fare'], bins=5) # discretised 
df_bin

In [ ]:
# What do our Fare bins look like?
X.Fare.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
trf.fit_transform(X_train)
X_test_trf = trf.transform(X_test)

In [ ]:
trf.named_transformers_['second'].n_bins_

In [ ]:
trf.named_transformers_['first'].n_bins_

In [ ]:
# Range of the bins coulms
trf.named_transformers_['first'].bin_edges_

In [ ]:
# Range of the bins coulms
trf.named_transformers_['second'].bin_edges_

In [ ]:
#df_bin['Fare'] = pd.cut(train['Fare'], bins=5) # discretised 
output_age = pd.cut(x=X_train['Age'],bins=trf.named_transformers_['first'].bin_edges_[0].tolist())
output_fare = pd.cut(x=X_train['Fare'],bins=trf.named_transformers_['second'].bin_edges_[0].tolist())

In [ ]:
output_age.isnull().sum()

In [ ]:
output_fare.isnull().sum()

In [ ]:
output_fare.value_counts().sum()

In [ ]:
def discretize(bins,strategy):
    kbin_age = KBinsDiscretizer(n_bins=bins,encode='ordinal',strategy=strategy)
    kbin_fare = KBinsDiscretizer(n_bins=bins,encode='ordinal',strategy=strategy)
    
    trf = ColumnTransformer([
        ('first',kbin_age,[0]),
        ('second',kbin_fare,[1])
    ])
    
    X_trf = trf.fit_transform(X)
    print(np.mean(cross_val_score(DecisionTreeClassifier(),X,y,cv=10,scoring='accuracy')))
    
    plt.figure(figsize=(14,4))
    plt.subplot(121)
    plt.hist(X['Age'])
    plt.title("Before")

    plt.subplot(122)
    plt.hist(X_trf[:,0],color='red')
    plt.title("After")

    plt.show()
    
    plt.figure(figsize=(14,4))
    plt.subplot(121)
    plt.hist(X['Fare'])
    plt.title("Before")

    plt.subplot(122)
    plt.hist(X_trf[:,1],color='red')
    plt.title("Fare")

    plt.show()    

In [ ]:
discretize(10,'quantile')

In [ ]:
def_num

In [ ]:
def_num.info()

In [ ]:
def_num['Parch'].value_counts()

In [ ]:
def_num['SibSp'].value_counts()

# Combining Parch and SibSp

* Its a right skew data

In [ ]:
def_num['Family'] = def_num['SibSp'] + def_num['Parch'] + 1
def_num['Family']

# Appling Bins to the data

* Converting Numerical data to Categorical data

In [ ]:
def fam_cat(x):
    if x == 1:
        return 'Alone'
    elif x > 1 and x <= 4:
        return 'Small Family'
    else:
        return 'Large Family'

In [ ]:
def_num['Family'].value_counts()